Hello dis is test

In [2]:
from Inner_Speech_Dataset.Python_Processing.Data_extractions import  Extract_data_from_subject

datatype = "EEG"
root_dir = "../dataset"
subjectNr = 1

data, label = Extract_data_from_subject(root_dir, subjectNr, datatype)

In [3]:
print(type(data[10])) # [39869     3     0     1]  I think it means this:  [unknown     class?     contaminated?     subjectNr]
print(data.shape)
print(data[0].shape)
# 500 trials

<class 'numpy.ndarray'>
(500, 128, 1153)
(128, 1153)


In [4]:
#!ls -la .. # ! to write bash commands

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class EEGNet(nn.Module):
    def __init__(self):
        super(EEGNet, self).__init__()
        self.f1 = 16
        self.d = 4
        self.f2 = self.d*self.f1

        # Layer 1
        self.conv1 = nn.Conv2d(in_channels=1, out_channels = self.f1, kernel_size = (1, 128), padding = 0, bias = False) # 128 = half sampling rate, reduces to 2Hz, 16 is probably F1, the number of filters (output channels)
        self.batchnorm1 = nn.BatchNorm2d(self.f1, False)

        self.conv2 = nn.Conv2d(1, self.d, (128, 1), bias = False) # row, column] 128 = number of sensors for the depth wise convolution
        # I'm missing a maxnorm regilarization here.
        # Add dropout in the forward func.
        self.batchnorm2 = nn.BatchNorm2d(self.d, False) # Not sure this is the right number of features.
        self.elu = nn.ELU()
        self.avgPool1 = nn.AvgPool2d((1,4)) # Reduces sampling rate to 64hz

        # Block 2

        # Separable convolution
        #self.depthwise = nn.Conv2d(nin, nin, kernel_size = (1,32), padding=1, groups=nin, bias = False) # Captures 500ms of data at sampling rate 64Hz (32 Hz is 500ms of that)
        #self.pointwise = nn.Conv2d(nin, self.f2, kernel_size = 1)


    def forward(self, x):
        res = self.conv1(x)
        res = self.batchnorm1(res)
        res = self.conv2(res)
        res = self.batchnorm2(res)
        # res = F.dropout(0.25)
        res = self.elu(res)
        res = self.avgPool1(res)
        res = F.dropout(0.25)
        return res


In [20]:
network = EEGNet()
d = torch.tensor(data[0])
network(d)


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [16, 1, 1, 128], but got 2-dimensional input of size [128, 1153] instead